# Google AlloyDB for PostgreSQL

> [Google Cloud AlloyDB for PostgreSQL](https://cloud.google.com/alloydb) 是专为要求最为严苛的企业工作负载打造的全托管 `PostgreSQL` 兼容数据库服务。`AlloyDB` 将 `Google Cloud` 的优势与 `PostgreSQL` 相结合，实现了卓越的性能、扩展性和可用性。利用 `AlloyDB` 的 Langchain 集成，扩展您的数据库应用程序，构建由 AI 驱动的体验。

本 Notebook 将介绍如何使用 `Google Cloud AlloyDB for PostgreSQL` 通过 `AlloyDBChatMessageHistory` 类存储聊天消息历史记录。

在 [GitHub](https://github.com/googleapis/langchain-google-alloydb-pg-python/) 上了解该软件包的更多信息。

[![在 Colab 中打开](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-alloydb-pg-python/blob/main/docs/chat_message_history.ipynb)

## 开始之前

要运行此笔记本，您需要执行以下操作：

* [创建 Google Cloud 项目](https://developers.google.com/workspace/guides/create-project)
* [启用 AlloyDB API](https://console.cloud.google.com/flows/enableapi?apiid=alloydb.googleapis.com)
* [创建 AlloyDB 实例](https://cloud.google.com/alloydb/docs/instance-primary-create)
* [创建 AlloyDB 数据库](https://cloud.google.com/alloydb/docs/database-create)
* [向数据库添加 IAM 数据库用户](https://cloud.google.com/alloydb/docs/manage-iam-authn) (可选)

### 🦜🔗 库安装
集成功能位于其自身的 `langchain-google-alloydb-pg` 包中，因此我们需要安装它。

In [ ]:
%pip install --upgrade --quiet langchain-google-alloydb-pg langchain-google-vertexai

**仅限 Colab：** 取消注释以下单元格以重启内核或使用按钮重启内核。对于 Vertex AI Workbench，您可以使用顶部的按钮重启终端。

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 🔐 认证
使用此笔记本中登录的 IAM 用户向 Google Cloud 进行认证，以访问你的 Google Cloud 项目。

* 如果你正在使用 Colab 运行此笔记本，请使用下面的代码单元并继续。
* 如果你正在使用 Vertex AI Workbench，请在此处查看设置说明：[https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env)。

In [ ]:
from google.colab import auth

auth.authenticate_user()

### ☁ 设置您的 Google Cloud 项目
设置您的 Google Cloud 项目，以便您可以在此笔记本中利用 Google Cloud 资源。

如果您不知道您的项目 ID，可以尝试以下方法：

* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 请参阅支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
# @markdown Please fill in the value below with your Google Cloud project ID and then run the cell.

PROJECT_ID = "my-project-id"  # @param {type:"string"}

# Set the project id
!gcloud config set project {PROJECT_ID}

### 💡 API 启用
`langchain-google-alloydb-pg` 包要求您在 Google Cloud 项目中[启用 AlloyDB Admin API](https://console.cloud.google.com/flows/enableapi?apiid=alloydb.googleapis.com)。

In [ ]:
# enable AlloyDB API
!gcloud services enable alloydb.googleapis.com

## 基本用法

### 设置 AlloyDB 数据库值
在 [AlloyDB 集群页面](https://console.cloud.google.com/alloydb?_ga=2.223735448.2062268965.1707700487-2088871159.1707257687) 查找你的数据库值。

In [ ]:
# @title Set Your Values Here { display-mode: "form" }
REGION = "us-central1"  # @param {type: "string"}
CLUSTER = "my-alloydb-cluster"  # @param {type: "string"}
INSTANCE = "my-alloydb-instance"  # @param {type: "string"}
DATABASE = "my-database"  # @param {type: "string"}
TABLE_NAME = "message_store"  # @param {type: "string"}

### AlloyDBEngine 连接池

建立 AlloyDBChat Message History 内存存储的要求之一是提供一个 `AlloyDBEngine` 对象。`AlloyDBEngine` 配置了到你的 AlloyDB 数据库的连接池，从而能够从你的应用程序成功建立连接，并遵循行业最佳实践。

要使用 `AlloyDBEngine.from_instance()` 创建一个 `AlloyDBEngine`，你只需要提供 5 个要素：

1. `project_id`：AlloyDB 实例所在 Google Cloud 项目的项目 ID。
1. `region`：AlloyDB 实例所在的区域。
1. `cluster`：AlloyDB 集群的名称。
1. `instance`：AlloyDB 实例的名称。
1. `database`：要连接到 AlloyDB 实例上的数据库的名称。

默认情况下，将使用 [IAM 数据库身份验证](https://cloud.google.com/alloydb/docs/manage-iam-authn) 作为数据库身份验证方法。此库使用源自环境的 [应用程序默认凭据 (ADC)](https://cloud.google.com/docs/authentication/application-default-credentials) 所属的 IAM 主体。

可选地，也可以使用[内置数据库身份验证](https://cloud.google.com/alloydb/docs/database-users/about)，即使用用户名和密码来访问 AlloyDB 数据库。只需向 `AlloyDBEngine.from_instance()` 提供可选的 `user` 和 `password` 参数即可：

* `user`：用于内置数据库身份验证和登录的数据库用户。
* `password`：用于内置数据库身份验证和登录的数据库密码。

In [ ]:
from langchain_google_alloydb_pg import AlloyDBEngine

engine = AlloyDBEngine.from_instance(
    project_id=PROJECT_ID,
    region=REGION,
    cluster=CLUSTER,
    instance=INSTANCE,
    database=DATABASE,
)

### 初始化表
`AlloyDBChatMessageHistory` 类需要一个具有特定架构的数据库表才能存储聊天消息历史记录。

`AlloyDBChatEngine` 引擎有一个名为 `init_chat_history_table()` 的辅助方法，可用于为您创建具有proper schema 的表。

In [ ]:
engine.init_chat_history_table(table_name=TABLE_NAME)

### AlloyDBChatMessageHistory

要初始化 `AlloyDBChatMessageHistory` 类，您只需提供 3 项：

1. `engine` - `AlloyDBEngine` 引擎的实例。
1. `session_id` - 一个唯一的标识符字符串，用于指定会话 ID。
1. `table_name` : AlloyDB 数据库中用于存储聊天消息历史记录的表的名称。

In [ ]:
from langchain_google_alloydb_pg import AlloyDBChatMessageHistory

history = AlloyDBChatMessageHistory.create_sync(
    engine, session_id="test_session", table_name=TABLE_NAME
)
history.add_user_message("hi!")
history.add_ai_message("whats up?")

In [ ]:
history.messages

#### 清理
当某个会话的历史记录已过时可以被删除时，可以按以下方式进行。

**注意：** 一旦删除，数据将不再存储在 AlloyDB 中，且将永久丢失。

In [ ]:
history.clear()

## 🔗 链式调用

我们可以轻松地将此消息历史记录类与 [LCEL Runnables](/docs/how_to/message_history) 组合起来。

为此，我们将使用 [Google 的 Vertex AI 聊天模型](/docs/integrations/chat/google_vertex_ai_palm) 之一，该模型要求您在 Google Cloud 项目中 [启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

In [ ]:
# enable Vertex AI API
!gcloud services enable aiplatform.googleapis.com

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_google_vertexai import ChatVertexAI

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = prompt | ChatVertexAI(project=PROJECT_ID)

In [ ]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: AlloyDBChatMessageHistory.create_sync(
        engine,
        session_id=session_id,
        table_name=TABLE_NAME,
    ),
    input_messages_key="question",
    history_messages_key="history",
)

In [ ]:
# This is where we configure the session id
config = {"configurable": {"session_id": "test_session"}}

In [ ]:
chain_with_history.invoke({"question": "Hi! I'm bob"}, config=config)

In [ ]:
chain_with_history.invoke({"question": "Whats my name"}, config=config)